# DVC Getting Started

В этом ноутбуке показаны основы работы с DVC на примере двух пользователей. 
Первый пользователь настраивает DVC и пайплайн.
Второй пользователь воспроизводит результаты первого.

# Подготовка

Чтобы освоить dvc нам понадобится несколько репозиториев:
- один git-репозиторий, который будет выполнять роль remote/origin
- 2 git для юзера 1 и юзера 2.
- 1 dvc репозиторий, который будет выполнять роль remote

Настроим перечисленные репозитории:

In [1]:
import os
from os.path import expanduser

# Библиотека для репозиториев 
ROOT_DIR = os.path.join(expanduser("~"), 'tmp_dvc')
GIT_REMOTE = os.path.join(ROOT_DIR, 'git_remote')
DVC_REMOTE = os.path.join(ROOT_DIR, 'dvc_remote')
USER1_DIR = os.path.join(ROOT_DIR, 'user1')
USER2_DIR = os.path.join(ROOT_DIR, 'user2')

os.makedirs(GIT_REMOTE)
os.makedirs(DVC_REMOTE)
os.makedirs(USER1_DIR)
os.makedirs(USER2_DIR)

In [2]:
# Initialize git remote repository
!git init --bare $GIT_REMOTE

Initialized empty Git repository in /home/neuro/tmp_dvc/git_remote/


# Первый пользователь

Инициализируем репозиторий для первого пользователя

In [3]:
!git clone $GIT_REMOTE $USER1_DIR

Cloning into '/home/neuro/tmp_dvc/user1'...
done.


## Создание файлов для обучения

У первого пользователь создадим датасет для обучения.
Это будет CSV с датасетом вин (https://archive.ics.uci.edu/ml/datasets/wine+quality)
Позже мы добавим этот файл под контроль DVC.

In [4]:
csv_path = os.path.join(USER1_DIR, 'winequality-red.csv')
!cp 'winequality-red.csv' $csv_path

Создадим в репозитории первого пользователя скрипт, который будет обучать модель:

In [5]:
script_path = os.path.join(USER1_DIR, 'train_model.py')
!cp 'train_model.py' $script_path

## Настройка DVC

In [6]:
# Перейти в каталог первого пользователя
%cd $USER1_DIR

# Инициализация dvc в репозитории
!dvc init

/home/neuro/tmp_dvc/user1
Adding '.dvc/state' to '.dvc/.gitignore'.
Adding '.dvc/lock' to '.dvc/.gitignore'.
Adding '.dvc/config.local' to '.dvc/.gitignore'.
Adding '.dvc/updater' to '.dvc/.gitignore'.
Adding '.dvc/updater.lock' to '.dvc/.gitignore'.
Adding '.dvc/state-journal' to '.dvc/.gitignore'.
Adding '.dvc/state-wal' to '.dvc/.gitignore'.
Adding '.dvc/cache' to '.dvc/.gitignore'.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|              https://dvc.org/doc/user-guide/analytics               |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: https://

Если выполнить команду `git status`, то мы увидим, что появился каталог `.dvc` с файлами `config` и `.gitignore`

In [7]:
# Показать что изменилось в репозитории
!git status

On branch master

Initial commit

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

	new file:   .dvc/.gitignore
	new file:   .dvc/config

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	train_model.py
	winequality-red.csv



Для работы dvc необходим удаленный репозиторий для хранения всех данных. Добавим его:

In [8]:
!dvc remote add myremote $DVC_REMOTE

Добавим наш датасет под контроль DVC:

In [9]:
!dvc add 'winequality-red.csv'

Adding 'winequality-red.csv' to '.gitignore'.
Saving 'winequality-red.csv' to '.dvc/cache/2d/aeecee174368f8a33b82c8cccae3a5'.
Saving information to 'winequality-red.csv.dvc'.

To track the changes with git run:

	git add .gitignore winequality-red.csv.dvc


После `git add` появится новый файл 'winequality-red.csv.dvc', а csv-файл будет добавлен в `.gitignore`. При работе с DVCS именно файлы с расширением `*.dvc` должны находиться под контролем git, а реальные файлы должны храниться в кэше DVC и хранилище DVC.

Сохраним наш прогресс:

In [10]:
!git add .
!git commit -m "Initialized dataset"

[master (root-commit) 3f5e0a4] Initialized dataset
 5 files changed, 72 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .gitignore
 create mode 100644 train_model.py
 create mode 100644 winequality-red.csv.dvc


Убедимся, что в remote-репозитории dvc наши данные отсутствуют:

In [11]:
!dvc status -r myremote

Preparing to collect status from /home/neuro/tmp_dvc/dvc_remote
[##############################] 100% Collecting information
new:                winequality-red.csv


Запушим наш прогресс:

In [12]:
# "dvc push" обязателен! Иначе у пользователей не будет возможности получить файлы под контролем DVC!!!
!dvc push -r myremote

Preparing to upload data to '/home/neuro/tmp_dvc/dvc_remote'
Preparing to collect status from /home/neuro/tmp_dvc/dvc_remote
[##############################] 100% Collecting information
[##############################] 100% Analysing status.
[##############################] 100% winequality-red.csv


In [13]:
!git push

Counting objects: 8, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (7/7), done.
Writing objects: 100% (8/8), 1.25 KiB | 0 bytes/s, done.
Total 8 (delta 0), reused 0 (delta 0)
To /home/neuro/tmp_dvc/git_remote
 * [new branch]      master -> master


## Обновление файлов под контролем DVC

Рано или поздно нам понадобиться обновлять файлы под DVC-контролем.

Покажем как это делается:

In [14]:
# Дополним наш датасет новыми данными:
!echo '6;0.31;0.47;3.6;0.067;18;42;0.99549;3.39;0.66;11;6' >> 'winequality-red.csv'

dvc заметит, что файлы изменились:

In [15]:
!dvc status

winequality-red.csv.dvc:
	changed outs:
		modified:           winequality-red.csv


Добавим изменения в кэш DVC:

In [16]:
!dvc add winequality-red.csv

Saving 'winequality-red.csv' to '.dvc/cache/9e/55c66554e24691183fa3a2e5ae3c9d'.
Saving information to 'winequality-red.csv.dvc'.

To track the changes with git run:

	git add winequality-red.csv.dvc


**ВАЖНО!!!**

Обновлять файлы нужно осторожно. Если в DVC изменяли параметр cache.type на значения hardlink и/или symlink (значения по умолчанию - reflink,copy), то нужно сначала убрать файл из под контроля DVC. Подробнее см. https://dvc.org/doc/user-guide/update-tracked-file

## Создание DVC-пайплайнов

В этом разделе мы:
- Обучим нашу модель
- Дадим другим пользователям возможность воспроизвести результаты экспериментов без переобучения всей модели

In [17]:
import sklearn

In [18]:
%%bash
dvc run \
    -f train_model.dvc \
    -d train_model.py -d 'winequality-red.csv' \
    -o 'mymodel-pickle.sav' \
    -M auc.metric \
    python train_model.py \
        --input_csv_path 'winequality-red.csv' \
        --output_checkpoint_path 'mymodel-pickle.sav' \
        --output_metrics_path 'auc.metric'

Running command:
	python train_model.py --input_csv_path winequality-red.csv --output_checkpoint_path mymodel-pickle.sav --output_metrics_path auc.metric
Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')
Training model...
Evaluating metrics...
R-score is 0.3323662048467976
Adding 'mymodel-pickle.sav' to '.gitignore'.
Output 'auc.metric' doesn't use cache. Skipping saving.
Saving 'mymodel-pickle.sav' to '.dvc/cache/7d/4bdbea2fbe202aa6f8a6d33d967991'.
Saving information to 'train_model.dvc'.

To track the changes with git run:

	git add .gitignore train_model.dvc


Посмотрим, что изменилось у нас:

In [19]:
!git status

On branch master
Your branch is up-to-date with 'origin/master'.
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   .gitignore
	modified:   winequality-red.csv.dvc

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	auc.metric
	train_model.dvc

no changes added to commit (use "git add" and/or "git commit -a")


Сохраним наш прогресс:

In [20]:
%%bash
git add .
git commit -m "Training pipeline"

# Запушить в DVC:
dvc push -r myremote

# Запушить в git:
git push

[master 7898874] Training pipeline
 4 files changed, 28 insertions(+), 6 deletions(-)
 create mode 100644 auc.metric
 create mode 100644 train_model.dvc
Preparing to upload data to '/home/neuro/tmp_dvc/dvc_remote'
Preparing to collect status from /home/neuro/tmp_dvc/dvc_remote
[##############################] 100% Collecting information
[##############################] 100% Analysing status.
(1/2): [##############################] 100% mymodel-pickle.sav
(2/2): [##############################] 100% winequality-red.csv


To /home/neuro/tmp_dvc/git_remote
   3f5e0a4..7898874  master -> master


# Второй пользователь

Давайте воспроизведем построенный пайплайн у второго пользователя

Перейти в каталог второго пользователя:

In [21]:
%cd $USER2_DIR

/home/neuro/tmp_dvc/user2


Клонируем репозиторий

In [22]:
!git clone $GIT_REMOTE $USER2_DIR

Cloning into '/home/neuro/tmp_dvc/user2'...
done.


Получаем данные из dvc:

In [23]:
!dvc pull -r myremote

Preparing to download data from '/home/neuro/tmp_dvc/dvc_remote'
Preparing to collect status from /home/neuro/tmp_dvc/dvc_remote
[##############################] 100% Collecting information
[##############################] 100% Analysing status.
(1/2): [##############################] 100% mymodel-pickle.sav
(2/2): [##############################] 100% winequality-red.csv
[##############################] 100% Checkout finished!v


При выполнении `dvc pull` команда `dvc` проходить по файлам `*.dvc` и по прописанным внутри них хэшам понимает какие файлы нужно скачать.

Убедимся, что у нас актуальная версия данных:

In [24]:
!dvc status

Pipeline is up to date. Nothing to reproduce.


Воспроизведем результаты первого пользователя:

In [25]:
!dvc repro train_model.dvc

Stage 'winequality-red.csv.dvc' didn't change.
Stage 'train_model.dvc' didn't change.
Pipeline is up to date. Nothing to reproduce.


DVC убедился, что все текущие данные актуальны и переобучать модель не надо. 
Если бы требовалось переобучить модель, то dvc бы запустил скрипт для обучения.